# Coursera Project Notebook

**This notebook is to be used for the project Capstone**



In [ ]:
import pandas as pd
import numpy as np

In [ ]:
print('Hello Capstone Project Course!')

In [ ]:
!pip3 install beautifulsoup4
!pip3 install lxml
!pip3 install requests
!pip3 install html5lib
#from bs4 import BeautifulSoup
#import requests

#source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
#soup = BeautifulSoup(source, 'lxml')
#print(soup.prettify())

# Week 3 assignment

**Question 1, to get the table from wikipedia and then put it in dataframe**    
  
+ step 1: read the excel file and then put it in df 
+ step 2: filter out the rows which with Borough == Not Assigned 
+ step 3: assign the value for Neighbourhood which is 'Not assigned'  
+ step 4: merge the rows with same postcode and Borough   

In [98]:
df = pd.read_excel('post_code_ca_M.xlsx')

df = df.loc[df.Borough != 'Not assigned']

df['Neighbourhood'] = df.apply(lambda x: x.Neighbourhood if x.Neighbourhood != 'Not assigned' else x.Borough, axis = 1)

df_converted = df.groupby(['Postcode', 'Borough']).Neighbourhood.apply(lambda x: ', '.join(x)).reset_index()

df_converted.loc[df_converted.Borough.str.contains('Quee')]
df_converted.shape

(103, 3)

### Question 2: 

It seems the geocoder does workday
```
!pip3 install geocoder
import geocoder # import geocoder
def get_lat_long(postal_code): 
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude, longitude

df_converted['tude'] = df_converted.Postcode.apply(get_lat_long)
```

I decided to use the csv file to come up with the latitude and longitude

### The following code read the latitude and longitude csv file  
Then merge the df_converted with the df_lat to get the columns latitude and longitude, the final result is df_final

In [100]:
df_lat = pd.read_csv('http://cocl.us/Geospatial_data')

df_final = df_converted.merge(df_lat, how = 'left', right_on = 'Postal Code', left_on = 'Postcode')

df_final.drop(columns = ['Postal Code'], inplace = True)

df_final.shape

(103, 5)

## Question 3: 
To show the neighbourhood in folium map and use k-mean to clustering the data

In [ ]:
!pip install folium
import folium

In [104]:
df_final = df_final.loc[df_final.Borough.str.contains('Toronto')]
df_final

,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In [102]:

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[43.657952, -79.387383], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_final['Latitude'], df_final['Longitude'], df_final['Borough'], df_final['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [103]:
CLIENT_ID = '0BGEG04QCVWHDJBCBD3RSACRZROUW53B4Z1Y315FMQM0HXXO' # your Foursquare ID
CLIENT_SECRET = 'QNC2WNXITMHCX0CR5NGX2JTOPTC5UICRHDF0SHKZZTJCEYJJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0BGEG04QCVWHDJBCBD3RSACRZROUW53B4Z1Y315FMQM0HXXO
CLIENT_SECRET:QNC2WNXITMHCX0CR5NGX2JTOPTC5UICRHDF0SHKZZTJCEYJJ
